# Correlations

**Imports** - *External*

In [1]:
%matplotlib widget
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import scipy
from scipy.stats import pearsonr
import matplotlib.pyplot as plt

*Internal*

In [2]:
from Merge import Merge # This file already has the data converted from string to numeric

**There are some errors while calculating the correlation, these are shown as warnigns. The next line will ignore these errors**

In [3]:
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

**The path used to save the correlation file is:**

In [4]:
path = "./dataFiles/corr"

**Fnction used to calculate correlation between two columns where they both have some sort of value:**

This function is using Pearson product-moment correlation coefficients:

\begin{equation}
    R_{ij} = \frac{ C_{ij} } { \sqrt{ C_{ii} * C_{jj} } }
\end{equation}


In [5]:
def getCorr(x, y):
    xData = []
    yData = []
    for i in range(len(x)): # Both columns must have same size
        if x[i] != '' and y[i] != '':
            xData.append(float(x[i]))
            yData.append(float(y[i]))
    xData = np.array(xData)
    yData = np.array(yData)
    return np.corrcoef(xData, yData)[0, 1]

Load the data:

In [6]:
try:
    correlations = pd.read_pickle(path+'.pickle')
except:
    data = Merge().data

    dataLabels = data.columns.values
    dataNumpy = data.to_numpy()

    correlations = pd.DataFrame(index=dataLabels, columns=dataLabels)

    for i in range(len(dataLabels)):
        correlations.at[dataLabels[i], dataLabels[i]] = 1
        
        # The next two lines ignore all warnings within this category
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=RuntimeWarning)
            
            for j in range(i+1, len(dataLabels)):
                corr = getCorr(dataNumpy[:, i], dataNumpy[:, j])
                correlations.at[dataLabels[i], dataLabels[j]] = correlations.at[dataLabels[j], dataLabels[i]] = 0 if np.isnan(corr) else abs(corr)
            
    correlations.to_pickle(path+'.pickle')
    correlations.to_csv(path+'.csv') # CSV is easy to see and can be opened in Excel

Here we will proceed tho the configuration of the **seaborn** heatmap.

In [7]:
aux = sns.color_palette("coolwarm", as_cmap=True)

In [15]:
plt.close()

sns.heatmap(np.ma.filled(correlations.astype(float), np.nan), annot=True, annot_kws={"size": 7}, xticklabels=correlations.columns, yticklabels=correlations.index, cmap=aux)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [9]:
def getCorrP(x, y):
    xData = []
    yData = []
    for i in range(len(x)): # Both columns must have same size
        if x[i] != '' and y[i] != '':
            xData.append(float(x[i]))
            yData.append(float(y[i]))
    if len(xData) > 1:
        return pearsonr(xData, yData)
    return 0, 0
    

In [10]:
data = Merge().data

dataLabels = data.columns.values
dataNumpy = data.to_numpy()

correlations = pd.DataFrame(index=dataLabels, columns=dataLabels)

pValues = pd.DataFrame(index=dataLabels, columns=dataLabels)

tableCorrP = pd.DataFrame(columns=["Labels", "Correlation", "p_Value"])
#getCorrP(dataNumpy[:, 2], dataNumpy[:,5])



for i in range(len(dataLabels)):
    correlations.at[dataLabels[i], dataLabels[i]] = 1
    pValues.at[dataLabels[i], dataLabels[i]] = 1
    for j in range(i+1, len(dataLabels)):
        p_r, p_p = getCorrP(dataNumpy[:, i], dataNumpy[:, j])
        correlations.at[dataLabels[i], dataLabels[j]] = correlations.at[dataLabels[j], dataLabels[i]] = p_r
        pValues.at[dataLabels[i], dataLabels[j]] = pValues.at[dataLabels[j], dataLabels[i]] = p_p
        
        tableCorrP = tableCorrP.append(dict(zip(tableCorrP.columns,[f"{dataLabels[i]} -> {dataLabels[j]}", p_r, p_p])), ignore_index = True)
        
        


 
    
    



Ready!


In [11]:
print(tableCorrP.head(10))

                    Labels  Correlation       p_Value
0          AGE -> PTGENDER     0.103754  2.251946e-37
1          AGE -> PTEDUCAT    -0.057173  2.117660e-12
2             AGE -> CDRSB     0.074271  9.850352e-15
3            AGE -> ADAS11     0.093143  6.334783e-22
4            AGE -> ADAS13     0.125313  3.875457e-38
5            AGE -> ADASQ4     0.161879  1.710975e-63
6              AGE -> MMSE    -0.084669  1.919914e-18
7   AGE -> RAVLT_immediate    -0.203302  8.476848e-99
8    AGE -> RAVLT_learning    -0.119734  5.512737e-35
9  AGE -> RAVLT_forgetting     0.044505  4.970359e-06


In [16]:
plt.close()
sns.heatmap(np.ma.filled(correlations.astype(float), np.nan), annot=True, annot_kws={"size": 7}, xticklabels=correlations.columns, yticklabels=correlations.index, cmap=aux)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [17]:
plt.close()
sns.heatmap(np.ma.filled(pValues.astype(float), np.nan), annot=True, annot_kws={"size": 7}, xticklabels=pValues.columns, yticklabels=pValues.index, cmap=aux)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>